<a href="https://colab.research.google.com/github/omar1slam/Random-ML-Scripts/blob/master/MNIST_NN_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
import tensorflow as tf

tf.test.gpu_device_name()

TensorFlow 2.x selected.


'/device:GPU:0'

In [ ]:
import numpy as np
import random
from mlxtend.data import loadlocal_mnist
from PIL import Image as Img
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import matplotlib.pyplot as plt
from sklearn import preprocessing


Images , Labels = loadlocal_mnist(
        images_path='/content/drive/My Drive/train-images.idx3-ubyte', labels_path='/content/drive/My Drive/train-labels.idx1-ubyte')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
normalized_X = preprocessing.normalize(Images)
print(Labels.shape)
data = normalized_X.reshape(60000, 28, 28, 1)
Labels = np.reshape(Labels, (60000, 1))
print(np.shape(data))
image = np.asarray(data[7453])
#plt.imshow(image, cmap='gray')
print(Labels.shape)

(60000,)
(60000, 28, 28, 1)
(60000, 1)


In [ ]:
X_train = data[:50000]
X_test = data[50000:60000]
Y_train = Labels[:50000]
Y_test = Labels[50000:60000]
print(X_train.shape)

(50000, 28, 28, 1)


In [ ]:
def relu(Z):
    return max(0, Z)

In [ ]:
def sigmoid(Z):
    return np.exp(Z)/(1 + np.exp(Z))

In [ ]:
def relu_derivative(Z):
    if Z > 0:
        return 1
    else:
        return 0

In [ ]:
def sigmoid_derivative(Z):
    return sigmoid(Z) * (1 - sigmoid(Z))


In [ ]:
def generate_dimensions(layers, r):

  dims = list()

  for l in range(layers):
    dims.append(r)
    
  return dims

In [ ]:
def initialize_parameters(n_nodes):

    n_layers = len(n_nodes)
    parameters = dict()
    np.random.seed(1)

    for l in range(1, n_layers):
        parameters['W' + str(l)] =  random.uniform(-0.1, 0.1) #np.random.randn(n_nodes[l], n_nodes[l-1])*0.01
        parameters['b' + str(l)] =  random.uniform(-0.1, 0.1) #np.random.randn(n_nodes[l], 1)*0.01

    return parameters

In [ ]:
def forward_propagation(A_prev, W, b, activation_function):

    Z = np.dot(W, A_prev) + b
    if activation_function == 'relu':
        A_current = relu(Z)
    if activation_function == 'sigmoid':
        A_current = sigmoid(Z)
    storage = (A_prev, Z, W, activation_function)

    return A_current, storage

In [ ]:
def forward_model(X, n_nodes):

    parameters = initialize_parameters(n_nodes)
    storage = list()
    n_layers = len(parameters) // 2
    A = X

    for l in range(1, n_layers):
        A, values = forward_propagation(A, parameters['W' + str(l)], parameters['b' + str(l)], 'relu')
        storage.append(values)

    A, values = forward_propagation(A, parameters['W' + str(n_layers)], parameters['b' + str(n_layers)], 'sigmoid')
    storage.append(values)

    return A, storage

In [ ]:
def compute_cost(Y, A):

    m = Y.shape[1]
    cost = 1/m * np.sum((1/2) * (Y-A)**2)
    cost = np.squeeze(cost) 
    
    return cost

In [ ]:
def back_propagation(dA, storage):

    A, Z, W, activation_function = storage
    m = A.shape[1]

    if activation_function == 'relu':
        dZ = relu_derivative(Z) * dA
    if activation_function == 'sigmoid':
        dZ = sigmoid_derivative(Z) * dA

    dA_prev = np.dot(W, dZ)
    dW = 1/m * np.dot(A.T, dZ)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)

    return dA_prev, dW, db

In [ ]:
def backward_model(Y, AL, storage):

    n_layers = len(storage) // 4
    gradients = dict()
    dA = Y - AL

    for l in range(n_layers, 0):
        dA_prev, dW, db = back_propagation(dA, storage[l])
        dA = dA_prev
        gradients['dW' + str(l)] = dW
        gradients['db' + str(l)] = db

    return gradients

In [ ]:
def update_parameters(parameters, grads, learning_rate):
  
  n_layers = len(parameters) // 2

  for l in range(n_layers):
    parameters['W' + str(l)] =  parameters['W' + str(l)] - learning_rate * grads['dW' + str(l)]
    parameters['b' + str(l)] =  parameters['W' + str(l)] - learning_rate * grads['db' + str(l)]

    return parameters


In [ ]:
def model(X, Y, n_nodes, learning_rate = 0.0075, num_iterations = 3000):

  parameters = initialize_parameters(n_nodes)
  for i in range(num_iterations):
      A, storage = forward_model(X, n_nodes)
      cost = compute_cost(Y, A)
      grads = backward_model(Y, A, storage)
      parameters = update_parameters(parameters, grads, learning_rate)
    
  return parameters


**CNN**

In [ ]:
keras.backend.clear_session()

In [ ]:
with tf.device('/device:GPU:0'):
  for r in range(10, 51):
    print(r)
    model = keras.Sequential([
    keras.layers.Conv2D(r, (3, 3), strides=(1, 1), input_shape=(28, 28, 1)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Flatten(),
           keras.layers.Dense(10, activation='softmax'),               
           ])
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 64, verbose = 1)
    preds = model.evaluate(x = X_test, y = Y_test)
    if not r == 50:
      plt.plot(r, history.history['loss'][-1], 'bo')
      plt.plot(r, preds[0], 'go')
  plt.plot(r, history.history['loss'][-1], 'bo', label='training')
  plt.plot(r, preds[0], 'go', label='testing')
  plt.legend()
  plt.grid(True)
  plt.title("Objective Function")

NameError: ignored

In [ ]:
keras.backend.clear_session()

In [ ]:
for r in range(10, 51):
  print(r)
  model = keras.Sequential([
    keras.layers.Conv2D(r, (3, 3), strides=(1, 1), input_shape=(28, 28, 1)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Conv2D(r, (3, 3), strides=(2, 2)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Flatten(),
           keras.layers.Dense(10, activation='softmax'),               
           ])
  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 64, verbose = 0)
  preds = model.evaluate(x = X_test, y = Y_test)
  if not r == 50:
    plt.plot(r, history.history['loss'][-1], 'bo')
    plt.plot(r, preds[0], 'go')
plt.plot(r, history.history['loss'][-1], 'bo', label='training')
plt.plot(r, preds[0], 'go', label='testing')
plt.legend()
plt.grid(True)
plt.title("Objective Function")

In [ ]:
keras.backend.clear_session()

In [ ]:
for r in range(10, 51):
  print(r)
  model = keras.Sequential([
    keras.layers.Conv2D(r, (3, 3), strides=(1, 1), input_shape=(28, 28, 1)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.Conv2D(r, (3, 3), strides=(2, 2)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Conv2D(r, (3, 3), strides=(2, 2)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Flatten(),
           keras.layers.Dense(10, activation='softmax'),               
           ])
  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 64, verbose = 0)
  preds = model.evaluate(x = X_test, y = Y_test)
  if not r == 50:
    plt.plot(r, history.history['loss'][-1], 'bo')
    plt.plot(r, preds[0], 'go')
plt.plot(r, history.history['loss'][-1], 'bo', label='training')
plt.plot(r, preds[0], 'go', label='testing')
plt.legend()
plt.grid(True)
plt.title("Objective Function")

In [ ]:
  model = keras.Sequential([
    keras.layers.Conv2D(16, (3, 3), strides=(1, 1), input_shape=(28, 28, 1)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.Conv2D(32, (3, 3), strides=(2, 2)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Conv2D(64, (3, 3), strides=(2, 2)),
           keras.layers.BatchNormalization(axis = 3),
           keras.layers.Activation('relu'),
           keras.layers.MaxPooling2D((2,2)),
           keras.layers.Flatten(),
           keras.layers.Dense(10, activation='softmax'),               
           ])

In [ ]:
model.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
preds = model.evaluate(x = X_test, y = Y_test)

In [ ]:
print(np.argmax(model.predict(data[9000].reshape(1, 28, 28, 1))))
data = data.reshape(60000, 28, 28)
image = np.asarray(data[9000])
plt.imshow(image, cmap='gray')

**NN**

In [ ]:
X_train_nn = X_train.reshape((X_train.shape[0], -1))
X_test_nn =  X_test.reshape((X_test.shape[0], -1))
Y_train_nn =  Y_train.reshape((Y_train.shape[0], -1))
Y_test_nn =  Y_test.reshape((Y_test.shape[0], -1))
print(X_train_nn.shape)

In [ ]:
keras.backend.clear_session()

In [ ]:
for r in range(10, 51):
  print(r)
  model_nn = keras.Sequential([
    keras.layers.Dense(r, input_shape=(784,), activation='relu'),
    keras.layers.Dense(10, activation='softmax'),               
    ])
  model_nn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  history = model_nn.fit(x = X_train_nn, y = Y_train_nn, epochs = 20, batch_size = 16, verbose=0)
  preds = model_nn.evaluate(x = X_test_nn, y = Y_test_nn)
  if not r == 50:
    plt.plot(r, history.history['loss'][-1], 'bo')
    plt.plot(r, preds[0], 'go')
plt.plot(r, history.history['loss'][-1], 'bo', label='training')
plt.plot(r, preds[0], 'go', label='testing')
plt.legend()
plt.grid(True)
plt.title("Objective Function")

In [ ]:
keras.backend.clear_session()

In [ ]:
for r in range(10, 51):
  print(r)
  model_nn = keras.Sequential([
    keras.layers.Dense(r, input_shape=(784,), activation='relu'),
    keras.layers.Dense(r, activation='relu'),
    keras.layers.Dense(10, activation='softmax'),               
    ])
  model_nn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  history = model_nn.fit(x = X_train_nn, y = Y_train_nn, epochs = 20, batch_size = 16, verbose=0)
  preds = model_nn.evaluate(x = X_test_nn, y = Y_test_nn)
  if not r == 50:
    plt.plot(r, history.history['loss'][-1], 'bo')
    plt.plot(r, preds[0], 'go')
plt.plot(r, history.history['loss'][-1], 'bo', label='training')
plt.plot(r, preds[0], 'go', label='testing')
plt.legend()
plt.grid(True)
plt.title("Objective Function")

In [ ]:
keras.backend.clear_session()

In [ ]:
for r in range(10, 51):
  print(r)
  model_nn = keras.Sequential([
    keras.layers.Dense(r, input_shape=(784,), activation='relu'),
    keras.layers.Dense(r, activation='relu'),
    keras.layers.Dense(r, activation='relu'),
    keras.layers.Dense(10, activation='softmax'),               
    ])
  model_nn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  history = model_nn.fit(x = X_train_nn, y = Y_train_nn, epochs = 20, batch_size = 16, verbose=0)
  preds = model_nn.evaluate(x = X_test_nn, y = Y_test_nn)
  if not r == 50:
    plt.plot(r, history.history['loss'][-1], 'bo')
    plt.plot(r, preds[0], 'go')
plt.plot(r, history.history['loss'][-1], 'bo', label='training')
plt.plot(r, preds[0], 'go', label='testing')
plt.legend()
plt.grid(True)
plt.title("Objective Function")

In [ ]:
data = data.reshape(60000, 28, 28)
#print(data[9000].reshape(784,).shape)
#print(np.argmax(model_nn.predict(data[9000].reshape(1, 784))))
image = np.asarray(data[7])
plt.imshow(image, cmap='gray')